In [1]:
import numpy as np
import pandas as pd
import re
import os

CARDS_DATA_INPUT_FILE = "pokemon-tcg-data-master 1999-2023.csv"
DECKS_DATA_INPUT_FILE = "tournaments.csv"
OUTPUT_DIR = "output/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Import data

cards_data = pd.read_csv(CARDS_DATA_INPUT_FILE).fillna("none")
decks = pd.read_csv(DECKS_DATA_INPUT_FILE)

##############
# Clean data #
##############

# List of characters to remove
chars_to_remove = ["[", "]", "{", "}", "'", ","]
pattern = f"[{re.escape(''.join(chars_to_remove))}]"

cards_data["cleaned_attacks"] = cards_data["attacks"].str.replace(
    pattern, "", regex=True
)
cards_data["cleaned_abilities"] = cards_data["abilities"].str.replace(
    pattern, "", regex=True
)
cards_data["cleaned_rules"] = cards_data["rules"].str.replace(pattern, "", regex=True)




In [53]:
trainers = cards_data.loc[cards_data["supertype"] == "Trainer"]
pokemon = cards_data.loc[cards_data["supertype"] != "Trainer"]

In [81]:
years_to_regulation = {
 'standard_2024': ['G'], 
 'standard_2023': ['E', 'F'], 
 'standard_2022': ['D', 'E'], 
 'standard_2021': ['D'],
 'pre_2021': ['none']
}

trainer_power_levels = {}

valid_years = set(['standard_2024', 'standard_2023', 'standard_2022', 'standard_2021'])
decks['temp_years'] = decks['rotation_name'].apply(lambda year: year if year in valid_years else 'pre_2021')
for year_name in ['standard_2024', 'standard_2023', 'standard_2022', 'standard_2021', 'pre_2021']:
    # Keep only cards in decks from the correct year
    decks_year = decks.loc[decks['temp_years'] == year_name]
    
    # Keep only cards that were in the valid regulations for that year
    trainers_year = trainers.loc[trainers['regulationMark'].isin(years_to_regulation[year_name])]
    
    # Remove duplicates. Trainer cards can be de-duped by just removing duplicate name 
    trainers_year = trainers_year.drop_duplicates(subset=["name"])
    
    # Create a new DataFrame of trainer cards in tournament decks
    decks_year_only_trainers = decks_year.loc[decks_year['name_card'].isin(trainers_year['name'])]
    
    # Calculate the count of each trainer cards that were used in tournament decks
    card_counts = decks_year_only_trainers.groupby('name_card')[['amount_card']].sum()
    
    # Count the number of decks in that year
    decks_count = decks_year.drop_duplicates(subset=['name_player', 'id_tournament']).shape[0]
    
    # Compute power level
    power_levels = card_counts.rename(columns={'amount_card':'power_level'})/decks_count/4
    
    # Combine power level with the card data. Neither dfs should have any duplicate card names
    trainers_year = trainers_year.merge(power_levels.reset_index(), left_on='name', right_on='name_card', how='left')
    
    # Fill in missing power levels with 0
    trainers_year['power_level'] = trainers_year['power_level'].fillna(0)
    
    # Store in dictionary
    trainer_power_levels[year_name] = trainers_year

    # We only have tournament data up until 2023, so there is no valid power level data for 2024
    if year_name == 'standard_2024':
        trainer_power_levels[year_name].drop(columns='power_level', inplace=True) # No play rate data for 2024
    

## Non-trainer cards
Non-trainer cards are much harder to handle because there are duplicate card names. The only way to distinguish the cards is by using the IDs, which do not match conveniently. 

In [66]:
# The ids do not match between the two datasets, so we will need to do something suspicious...
def extract_numbers(s):
    # Pull out just the set number
    match = re.findall(r'\d+', s)
    return ''.join(match).strip('0') if match else None

In [ ]:
pokemon_power_levels = {}

for year_name in ['standard_2024', 'standard_2023', 'standard_2022', 'standard_2021', 'pre_2021']:
    # Keep only cards in decks from the correct year
    decks_year = decks.loc[decks['temp_years'] == year_name]
    
    # Keep only cards that were in the valid regulations for that year
    pokemon_year = pokemon.loc[pokemon['regulationMark'].isin(years_to_regulation[year_name])]

    # Create the feature which we will join on... this is a bit of a hack
    # There are also typos in the data which need to be fixed
    # Basically it is taking the card number from the set and the name of the card, making a mostly unique identifier
    pokemon_year['temp_id'] = pokemon_year['set_num'].apply(extract_numbers) + '-' + pokemon_year['name'].str.replace('Team Flare Hyper Gear', '').str.replace('Team Flare Gear', '').str.replace(' ', '').str.lower()
    decks_year['temp_id'] = decks_year['id_card'].apply(extract_numbers) + '-' + decks_year['name_card'].str.replace('Pokemon Card Gym Medal', 'heliolisk').str.replace('Palace Book', 'zubat').str.replace(' ', '').str.lower()

    # Calculate the count of each pokemon cards that were used in tournament decks
    card_counts = decks_year.groupby('temp_id')[['amount_card']].sum()
    
    # Count the number of decks in that year
    decks_count = decks_year.drop_duplicates(subset=['name_player', 'id_tournament']).shape[0]
    
    # Computer power level
    power_levels = card_counts.rename(columns={'amount_card':'power_level'})/decks_count/4
    
    # Merge on the temp_id. There should be no duplicates
    pokemon_year = pokemon_year.merge(power_levels.reset_index(), on='temp_id', how='left')
    
    # Fill in missing power levels with 0
    pokemon_year['power_level'] = pokemon_year['power_level'].fillna(0)
    
    # Store in dictionary
    pokemon_power_levels[year_name] = pokemon_year
    
    if year_name == 'standard_2024':
        pokemon_power_levels[year_name].drop(columns='power_level', inplace=True) # No play rate data for 2024


C:\Users\tcl17\AppData\Local\Temp\ipykernel_24872\737140450.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokemon_year['temp_id'] = pokemon_year['set_num'].apply(extract_numbers) + '-' + pokemon_year['name'].str.replace('Team Flare Hyper Gear', '').str.replace('Team Flare Gear', '').str.replace(' ', '').str.lower()
C:\Users\tcl17\AppData\Local\Temp\ipykernel_24872\737140450.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokemon_year['temp_id'] = pokemon_year['set_num'].apply(extract_numbers) + 

### Export

In [83]:
for name, df in trainer_power_levels.items():
    df.to_csv(f"{OUTPUT_DIR}{name}_trainer_power_level.csv", index=False)
    print(f"Saved {name}_trainer_power_level.csv")

for name, df in pokemon_power_levels.items():
    df.to_csv(f"{OUTPUT_DIR}{name}_pokemon_power_level.csv", index=False)
    print(f"Saved {name}_pokemon_power_level.csv")


Saved standard_2024_trainer_power_level.csv
Saved standard_2023_trainer_power_level.csv
Saved standard_2022_trainer_power_level.csv
Saved standard_2021_trainer_power_level.csv
Saved pre_2021_trainer_power_level.csv
Saved standard_2024_pokemon_power_level.csv
Saved standard_2023_pokemon_power_level.csv
Saved standard_2022_pokemon_power_level.csv
Saved standard_2021_pokemon_power_level.csv
Saved pre_2021_pokemon_power_level.csv
